In [1]:
import sys, os
import glob
cwd = os.getcwd()
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer 
from collections import Counter
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import reuters, stopwords
import re
import nltk
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
finviz_url = 'https://finviz.com/quote.ashx?t='
tickers = ['AMZN', 'GOOG', 'TSLA']
news_tables = {}
for ticker in tickers:
    url = finviz_url + ticker
    req = Request(url=url, headers={'user-agent': 'my-app'})
    response = urlopen(req)
    html = BeautifulSoup(response, features='html.parser')
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table

parsed_data = []
url_dict = {}
for ticker, news_table in news_tables.items():
    url_ls = []
    for row in news_table.findAll('tr'):

        if row.a is None:
            pass
        else:
            title = row.a.text
            date_data = row.td.text.split(' ')
            url_extraction = str(row)
            string_search = re.search('https(.+?)"', url_extraction)
            if string_search:
                extracted_url = string_search.group(1)
                final_url = 'http' + extracted_url
                url_ls.append(final_url)


        if len(date_data) == 1:
            time = date_data[0]
        else:
            date = date_data[0]
            time = date_data[1]

        parsed_data.append([ticker, date, time, title])

    ticker_url_list = pd.DataFrame(url_ls, columns = [ticker])
    url_dict[ticker] = ticker_url_list
df = pd.DataFrame(parsed_data, columns=['ticker', 'date', 'time', 'title'])

vader = SentimentIntensityAnalyzer()

f = lambda title: vader.polarity_scores(title)['compound']
df['compound'] = df['title'].apply(f)
df['date'] = pd.to_datetime(df.date).dt.date


In [3]:
item='AMZN'
testing = url_dict['AMZN']
list = testing[item].tolist()
for item in list:
    print(list)

['http://finance.yahoo.com/news/u-begins-antitrust-review-amazon-011542610.html', 'http://finance.yahoo.com/news/u-begins-antitrust-review-amazon-011104249.html', 'http://www.wsj.com/articles/ftc-investigating-amazon-deal-to-buy-one-medical-network-of-health-clinics-11662157363?siteid=yhoof2', 'http://finance.yahoo.com/news/amazon-amzn-stock-moves-0-214509401.html', 'http://www.barrons.com/articles/dan-niles-satori-fund-gambling-apple-stocks-51662069936?siteid=yhoof2', 'http://finance.yahoo.com/news/u-ftc-asks-amazon-one-211317136.html', 'http://www.fool.com/investing/2022/09/02/why-plug-power-shares-plunged-40-on-friday/?source=eptyholnk0000202&amp;utm_source=yahoo-host&amp;utm_medium=feed&amp;utm_campaign=article', 'http://finance.yahoo.com/video/rings-power-viewers-report-issues-201822535.html', 'http://finance.yahoo.com/news/amazon-closes-abandons-plans-dozens-155259786.html', 'http://www.fool.com/investing/2022/09/02/why-cheetah-stock-soared-friday-morning/?source=eptyholnk0000202

In [4]:
for item in url_dict:
    url_df = (url_dict[item])
    ticker_url_list = url_df[item].tolist()
    for url in ticker_url_list:
        req = Request(url=url, headers={'user-agent': 'my-app'})
        response = urlopen(req)
        html = BeautifulSoup(response, features='html.parser')
        text = html.get_text()
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = '\n'.join(chunk for chunk in chunks if chunk)

KeyboardInterrupt: 

In [5]:
top_record_count=20
sample_url = str(text)
strip_lines=sample_url.strip()
strip_lines[0:300]

"Why a Bearish Money Manager Likes Gambling Stocks and Is Ready to Dump Apple | Barron's\nWe've detected you are on Internet Explorer. For the best Barrons.com experience, please update to a modern browser.\nCHROME\nSAFARI\nFIREFOX\nWe've detected you are on Internet Explorer. For the best Barrons.com exp"

In [6]:
listli=strip_lines.split()
flat_list = listli.copy()
listli[0:7]

['Why', 'a', 'Bearish', 'Money', 'Manager', 'Likes', 'Gambling']

In [11]:
stop_words

['than',
 'yourself',
 'before',
 'it',
 'of',
 'an',
 "should've",
 'over',
 "aren't",
 'only',
 'any',
 'was',
 'while',
 'in',
 'herself',
 'yourselves',
 'he',
 'under',
 'has',
 "don't",
 "you'll",
 "haven't",
 'themselves',
 'they',
 'we',
 'again',
 'why',
 'off',
 "wasn't",
 'at',
 'and',
 'as',
 'him',
 'during',
 'have',
 "shouldn't",
 'shan',
 "hadn't",
 'had',
 'their',
 'if',
 "couldn't",
 "you're",
 "didn't",
 'mustn',
 'wouldn',
 'these',
 'o',
 'his',
 'until',
 'do',
 'up',
 'ma',
 'more',
 'here',
 'each',
 'ain',
 "won't",
 'but',
 'y',
 'didn',
 'shouldn',
 'so',
 "isn't",
 'hers',
 'into',
 'where',
 "hasn't",
 'not',
 "mightn't",
 'theirs',
 'by',
 'hadn',
 've',
 'ourselves',
 'further',
 'll',
 't',
 'very',
 'that',
 'most',
 'can',
 'myself',
 'against',
 'there',
 'wasn',
 'once',
 "shan't",
 'own',
 'other',
 'how',
 'your',
 'or',
 'she',
 'her',
 'the',
 'needn',
 're',
 's',
 'this',
 'who',
 'just',
 'when',
 'them',
 "weren't",
 'ours',
 "she's",
 'migh

In [12]:
lemmatizer = WordNetLemmatizer()
stop_words_list = set(stopwords.words('english'))
# stop_words_list = list(nltk_stop_words)
include_in_stopwords = ['ha']
remove_from_stopwords = ['all', 'below', 'being', 'too']
stop_words = [elem for elem in stop_words_list if elem not in remove_from_stopwords]
stop_words = stop_words+include_in_stopwords
regex = re.compile("[^a-zA-Z ]")
re_clean = regex.sub('', strip_lines)
words = word_tokenize(re_clean)
lem = [lemmatizer.lemmatize(word) for word in words]
output = [word.lower() for word in lem if word.lower() not in stop_words]
top_10 = dict(Counter(output).most_common(top_record_count))
# top_10_df = pd.DataFrame(list(top_10.items()), columns=['word', 'count'])
top_10

{'please': 4,
 'copy': 4,
 'use': 4,
 'niles': 4,
 'stock': 4,
 'bearish': 3,
 'money': 3,
 'manager': 3,
 'likes': 3,
 'gambling': 3,
 'stocks': 3,
 'ready': 3,
 'dump': 3,
 'personal': 3,
 'dow': 3,
 'jones': 3,
 'detected': 2,
 'internet': 2,
 'explorer': 2,
 'best': 2}

In [9]:
# def process_text(strip_lines):
#     sw = set(stopwords.words('english'))
#     regex = re.compile("[^a-zA-Z ]")
#     re_clean = regex.sub('', strip_lines)
#     words = word_tokenize(re_clean)
#     lem = [lemmatizer.lemmatize(word) for word in words]
#     output = [word.lower() for word in lem if word.lower() not in sw]
#     return output


# def word_counter(corpus): 
#     # Combine all articles in corpus into one large string
#     big_string = ' '.join(corpus)
#     processed = process_text(big_string)
#     top_10 = dict(Counter(processed).most_common(top_record_count))
#     return pd.DataFrame(list(top_10.items()), columns=['word', 'count'])

# results = word_counter(flat_list)
# print(results)
# # results.to_csv(f'output/{file_name}_word-count.csv')


In [10]:
# for url in url_ls:
#     req = Request(url=url, headers={'user-agent': 'my-app'})
#     response = urlopen(req)
#     html = BeautifulSoup(response, features='html.parser')
#     text = html.get_text()
#     lines = (line.strip() for line in text.splitlines())
#     chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
#     text = '\n'.join(chunk for chunk in chunks if chunk)